<a href="https://colab.research.google.com/github/kovzanok/dls-final-task/blob/main/2_dop_Triplet_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Большую часть кода обучения взял с уже написанного мной 2 основного задания (файл ArcFace.ipynb)

Качаем датасет, полученный для обучения модели классификации

In [1]:
!gdown 1bHLaSZ2frNjyK2hLTXTuPFlFRxfJDxwb

Downloading...
From (original): https://drive.google.com/uc?id=1bHLaSZ2frNjyK2hLTXTuPFlFRxfJDxwb
From (redirected): https://drive.google.com/uc?id=1bHLaSZ2frNjyK2hLTXTuPFlFRxfJDxwb&confirm=t&uuid=a1539203-dcb7-4df6-847b-ba03c9ab726d
To: /content/file.zip
100% 401M/401M [00:07<00:00, 53.0MB/s]


In [2]:
!unzip /content/file.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/stage_3_dataset/112660.jpg  
  inflating: /content/content/stage_3_dataset/129052.jpg  
  inflating: /content/content/stage_3_dataset/151389.jpg  
  inflating: /content/content/stage_3_dataset/118111.jpg  
  inflating: /content/content/stage_3_dataset/000174.jpg  
  inflating: /content/content/stage_3_dataset/013829.jpg  
  inflating: /content/content/stage_3_dataset/107408.jpg  
  inflating: /content/content/stage_3_dataset/112157.jpg  
  inflating: /content/content/stage_3_dataset/119239.jpg  
  inflating: /content/content/stage_3_dataset/120061.jpg  
  inflating: /content/content/stage_3_dataset/147590.jpg  
  inflating: /content/content/stage_3_dataset/037708.jpg  
  inflating: /content/content/stage_3_dataset/072505.jpg  
  inflating: /content/content/stage_3_dataset/100668.jpg  
  inflating: /content/content/stage_3_dataset/118256.jpg  
  inflating: /content/content/stage_3_dataset/121771.jpg  
  inf

In [3]:
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split, Subset, Sampler
from PIL import Image
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.models as models
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF
import cv2
from sklearn.preprocessing import LabelEncoder

import random
import os
import math
from itertools import combinations

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#Датасет

Загружаем файл со связью имени изображения и id человека

In [7]:
!gdown 1pmjLR8zU17IQTVWYZrzLU_-XR33f1RtJ

Downloading...
From: https://drive.google.com/uc?id=1pmjLR8zU17IQTVWYZrzLU_-XR33f1RtJ
To: /content/identity_CelebA.txt
100% 3.42M/3.42M [00:00<00:00, 31.5MB/s]


In [8]:
df = pd.read_csv('/content/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)

/tmp/ipython-input-8-1097142516.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('/content/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)


In [9]:
df.index.name = 'image_name'
df.rename(columns={1:'id'},inplace=True)

In [10]:
df.head()

,id
image_name,
000001.jpg,2880
000002.jpg,2937
000003.jpg,8692
000004.jpg,5805
000005.jpg,9295


In [11]:
def get_filenames_os(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

Получаем имя файлов из датасета, прошедшего выравнивание и кроп моделью из прошлого задания. Если бы использовался датасет CelebA полностью эта операция не нужна была бы, но нам нужны id только тех людей, фото которых мы обработали в прошлом задании.

In [12]:
filenames = get_filenames_os('/content/content/stage_3_dataset')

In [13]:
filtered_df = df.loc[filenames]
ids = filtered_df['id'].to_list()

In [15]:
id_dict = {} # словарь вида { id1: [img_name1, img_name2,...] }

for id in tqdm(ids):
    img_names = filtered_df[filtered_df['id']==id].index.to_list()
    id_dict[id] = img_names

  0%|          | 0/151949 [00:00<?, ?it/s]

In [16]:
class TripletDataset(Dataset):
    def __init__(self, img_dir, id_dict, transform = None):
        super().__init__()
        self.img_dir = img_dir
        self.id_dict = id_dict
        self.ids = list(id_dict.keys())
        self.transform = transform

    def __len__(self):
        return 10000

    def __getitem__(self, idx):
        pos_id, neg_id = np.random.choice(self.ids, replace=False, size=2)

        pos_imgs = self.id_dict[pos_id]

        while len(pos_imgs) < 2:
            pos_id = np.random.choice(self.ids, replace=False, size=1)[0]
            pos_imgs = self.id_dict[pos_id]

        neg_imgs = self.id_dict[neg_id]

        anchor_img, pos_img = np.random.choice(pos_imgs, replace=False, size=2)
        neg_img = np.random.choice(neg_imgs, size=1)[0]

        def load(img_name):
            path = os.path.join(self.img_dir, img_name)
            img = Image.open(path).convert("RGB")
            return self.transform(img) if self.transform else img

        anchor = load(anchor_img)
        positive = load(pos_img)
        negative = load(neg_img)

        return anchor, positive, negative

In [17]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=5, translate=(0.02, 0.02)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

all_ids = list(id_dict.keys())
random.shuffle(all_ids)

train_ids = all_ids[:int(0.8 * len(all_ids))]
val_ids = all_ids[int(0.8 * len(all_ids)):]

train_id_dict = {id_: id_dict[id_] for id_ in train_ids}
val_id_dict = {id_: id_dict[id_] for id_ in val_ids}

train_dataset = TripletDataset(img_dir='/content/content/stage_3_dataset', id_dict=train_id_dict, transform=transform)
val_dataset = TripletDataset(img_dir='/content/content/stage_3_dataset', id_dict=val_id_dict, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

#Модель

In [18]:
embedding_dim = 128
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, embedding_dim)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 203MB/s]


In [19]:
model.load_state_dict(torch.load('/content/drive/MyDrive/best_recognition_model_triplet.pth', weights_only=True, map_location=device)['model'])

<All keys matched successfully>

In [20]:
for name, param in model.named_parameters():
    if 'layer1' in name or 'layer2' in name or 'layer3' in name or 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [21]:
model = model.to(device)

#Обучение

##Расчет Recall@K метрики

In [22]:
from sklearn.metrics import pairwise_distances


@torch.no_grad()
def compute_embeddings_and_labels(model, dataset):
    model.eval()
    embeddings = []
    labels = []

    for id_, img_names in tqdm(dataset.id_dict.items()):
        for img_name in img_names:
            path = os.path.join(dataset.img_dir, img_name)
            img = Image.open(path).convert("RGB")

            if dataset.transform:
                img = dataset.transform(img)

            img = img.unsqueeze(0).to(device)
            emb = model(img)
            emb = F.normalize(emb, dim=1)
            embeddings.append(emb.cpu().numpy()[0])
            labels.append(id_)

    embeddings = np.array(embeddings)
    labels = np.array(labels)
    return embeddings, labels

def recall_at_k_gpu(embeddings, labels, k=1, batch_size=5000):
    """
    Compute Recall@k using GPU-accelerated pairwise distances in PyTorch.

    Args:
        embeddings: NumPy array of shape (n_samples, embedding_dim)
        labels: NumPy array of shape (n_samples,)
        k: Number of nearest neighbors to consider
        batch_size: Process queries in batches to save memory

    Returns:
        Recall@k score
    """
    # Convert to PyTorch tensors and move to GPU
    embeddings = torch.from_numpy(embeddings).float().to(device)
    labels_tensor = torch.from_numpy(labels).to(device)
    n = len(embeddings)
    hits = 0

    for i in tqdm(range(0, n, batch_size), desc="Processing batches"):
        # Get current batch of queries
        batch = embeddings[i:i+batch_size]

        # Compute distances between batch and all embeddings
        dists = torch.cdist(batch, embeddings)  # shape: (batch_size, n)

        # Mask self-distances
        rows = torch.arange(i, min(i+batch_size, n), device=device)
        dists[torch.arange(len(batch)), rows] = float('inf')

        # Get top-k nearest neighbors (excluding self)
        _, topk = torch.topk(dists, k, dim=1, largest=False)

        # Compare labels (all operations stay on GPU)
        batch_labels = labels_tensor[i:i+batch_size]
        neighbor_labels = labels_tensor[topk]  # shape: (batch_size, k)

        # Check if correct label is among neighbors
        hits += torch.any(batch_labels.unsqueeze(1) == neighbor_labels, dim=1).sum().item()

    return hits / n

##Функции обучения

In [23]:
def train_epoch(model, criterion, optimizer, dataloader):
    model.train()

    running_loss = 0.0

    for anchor, positive, negative in tqdm(dataloader, leave=False):
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        emb_a = model(anchor)
        emb_p = model(positive)
        emb_n = model(negative)

        emb_a = F.normalize(emb_a, dim=1)
        emb_p = F.normalize(emb_p, dim=1)
        emb_n = F.normalize(emb_n, dim=1)

        loss = criterion(emb_a, emb_p, emb_n)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    avg_loss = running_loss / len(dataloader)
    return avg_loss


In [24]:
@torch.no_grad()
def val_epoch(model, criterion, dataloader, recall_k_list=[1,5]):
    model.eval()
    running_loss = 0.0

    avg_loss = running_loss / len(dataloader)

    embeddings, labels = compute_embeddings_and_labels(model, val_dataset)
    recall_scores = {k: recall_at_k_gpu(embeddings, labels, k) for k in recall_k_list}

    return avg_loss, recall_scores


In [25]:
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=100, scheduler=None):
    history = {'train':[],'val':[]}
    best_recall = 0

    for epoch in tqdm(range(num_epochs)):

        train_loss = train_epoch(model, criterion, optimizer, train_loader)
        val_loss, recall_scores = val_epoch(model, criterion, val_loader)

        history['train'].append(train_loss)
        history['val'].append(val_loss)


        print(f"Epoch {epoch+1} — Train Loss: {train_loss:.4f} — Val Loss: {val_loss:.4f}")
        for k, val in recall_scores.items():
            print(f"Recall@{k}: {val:.4f}")

        if scheduler:
          scheduler.step(recall_scores[1])

        if recall_scores[1] > best_recall:
            best_recall = recall_scores[1]
            torch.save({
                'model': model.state_dict(),
            }, "/content/drive/MyDrive/best_recognition_model_triplet.pth")
            print("✅ Saved best model")


    return history

##Обучение

In [ ]:
criterion = nn.TripletMarginLoss(margin=0.2, p=2)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [ ]:
history = train(model, criterion, optimizer, train_loader, val_loader, scheduler=scheduler, num_epochs = 50)

##Semihard mining

Отфильтруем данный, чтобы на каждый класс было как минимум 2 изображения

In [26]:
filtered_id_dict = { id:imgs  for id, imgs in id_dict.items() if len(imgs) >= 2 }

In [27]:
semihard_df = filtered_df[filtered_df['id'].isin(filtered_id_dict.keys())]

In [28]:
class SemihardSampler(Sampler):
    def __init__(self, id_dict, P, K, batch_count):
        self.id_dict = id_dict
        self.P = P
        self.K = K
        self.batch_count = batch_count
        self.all_ids = list(id_dict.keys())

    def __iter__(self):
       for _ in range(self.batch_count):
            random_ids = random.sample(self.all_ids, self.P)
            batch = []
            for id in random_ids:
                images = self.id_dict[id]
                if len(images) >= self.K:
                    batch.extend(random.sample(images, self.K))
                else:
                    batch.extend(random.choices(images, k=self.K))  # если вдруг меньше
            yield batch

    def __len__(self):
        return self.batch_count


In [29]:
class SemihardTripletDataset(Dataset):
    def __init__(self, img_dir, df, transform=None):
        super().__init__()
        self.img_dir = img_dir
        self.transform = transform
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label, img_name = self.df[self.df.index == idx]["id"].values[0], idx
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label


In [30]:
all_ids = list(filtered_id_dict.keys())
random.shuffle(all_ids)

train_ids = all_ids[:int(0.8 * len(all_ids))]
val_ids = all_ids[int(0.8 * len(all_ids)):]

train_id_dict = {id_: id_dict[id_] for id_ in train_ids}
val_id_dict = {id_: id_dict[id_] for id_ in val_ids}

# Создаём train_dataset
train_dataset = SemihardTripletDataset(
    img_dir='/content/content/stage_3_dataset',
    df=semihard_df,
    transform=transform
)

In [31]:
# Самплер: P класса × K изображения каждого
semihard_sampler = SemihardSampler(id_dict=train_id_dict, P=8, K=4, batch_count=1000)

# DataLoader
train_loader = DataLoader(train_dataset, batch_sampler=semihard_sampler, num_workers=2)

In [32]:
def train_epoch(model, criterion, optimizer, dataloader, margin=0.2):
    model.train()
    running_loss = 0.0
    num_triplets = 0

    for images, labels in tqdm(dataloader, leave=False):
        images, labels = images.to(device), labels.to(device)

        with torch.no_grad():
            embeddings = model(images)
            embeddings = F.normalize(embeddings, dim=1)

        triplets = []

        # Перебираем все пары (anchor, positive)
        for i in range(len(labels)):
            for j in range(i + 1, len(labels)):
                if labels[i] == labels[j]:
                    anchor = embeddings[i]
                    positive = embeddings[j]

                    # Находим semi-hard negatives
                    d_ap = F.pairwise_distance(anchor.unsqueeze(0), positive.unsqueeze(0), p=2).item()

                    for k in range(len(labels)):
                        if labels[k] != labels[i]:
                            negative = embeddings[k]
                            d_an = F.pairwise_distance(anchor.unsqueeze(0), negative.unsqueeze(0), p=2).item()

                            if d_ap < d_an < d_ap + margin:
                                triplets.append((i, j, k))

        if not triplets:
            continue  # ничего не делаем, если нет триплетов

        # Используем реальные эмбеддинги для градиентов
        embeddings = model(images)
        embeddings = F.normalize(embeddings, dim=1)

        anc = torch.stack([embeddings[i] for (i, _, _) in triplets])
        pos = torch.stack([embeddings[j] for (_, j, _) in triplets])
        neg = torch.stack([embeddings[k] for (_, _, k) in triplets])

        loss = criterion(anc, pos, neg)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num_triplets += len(triplets)



    if num_triplets == 0:
        return float("nan")  # или float("nan")

    avg_loss = running_loss / len(dataloader)
    return avg_loss


In [33]:
criterion = nn.TripletMarginLoss(margin=0.2, p=2)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [ ]:
history = train(model, criterion, optimizer, train_loader, val_loader, scheduler=scheduler, num_epochs = 50)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1639 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/1639 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]

Epoch 4 — Train Loss: 0.0912 — Val Loss: 0.0000
Recall@1: 0.1712
Recall@5: 0.3746
✅ Saved best model


  0%|          | 0/1000 [00:00<?, ?it/s]

#Тестовый блок

In [ ]:
embeddings.shape

[ ]
recall_k_list = [1,5]
recall_scores = {k: recall_at_k_gpu(embeddings, labels, k) for k in recall_k_list}


(30141, 128)

In [ ]:
recall_k_list = [1,5]
recall_scores = {k: recall_at_k_gpu(embeddings, labels, k) for k in recall_k_list}

Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
recall_scores

{1: 0.05832586841843336, 5: 0.12388440993994891}

In [ ]:
from google.colab import files
files.download("/content/best_recognition_model_triplet.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>